### Import all the libraries 

In [ ]:
import tensorflow as tf   #importing tensorflow libraries
import matplotlib.pyplot as plt  #importing matplotlib to plotting graphs
from tensorflow import keras   #importing keras for neural network
import cv2
import os
import numpy as np
import pandas as pd
from deepface import DeepFace
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from keras.optimizers import RMSprop, Adam  
from keras.models import Sequential  # To create a plain stack of layers where each layer has one input tensor and one output tensor
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,BatchNormalization


In [ ]:
plt.imshow(image.load_img('C:/Users/Abhay/Final_Capstone/Train/1-HAPPY/1.jpg'))  #loading the image

### Generate training, validation, testing dataset, and image augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)       #initiate class for training 
validation_datagen = ImageDataGenerator(rescale=1./255)  #initiate class for validation
test_datagen = ImageDataGenerator(rescale=1./255)  #initiate class for testing
train_data = train_datagen.flow_from_directory(
        'C:/Users/Abhay/Final_Capstone/Train',  # this is the input directory
        target_size=(100, 100),  # all images will be resized to 148x148 . Since we cannot give multiple size images to neural network, we resize the images
        batch_size=100,           # the neural network will train up in batches of 2
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data, with images being resized to 148x148,trained in batches of 2 with binary labels.
validation_data = validation_datagen.flow_from_directory(
        'C:/Users/Abhay/Final_Capstone/Validation',
        target_size=(100, 100),
        batch_size=10,
        class_mode='binary')
# this is a similar generator, for test data, with images being resized to 148x148,trained in batches of 2 with binary labels.
test_data = test_datagen.flow_from_directory(
        'C:/Users/Abhay/Final_Capstone/Test',
        target_size=(100, 100),
        batch_size=10,
        class_mode='binary')
train_data.class_indices # cars are labeled as 0 and not cars are labeled as 1

### Building CNN

In [ ]:
INPUT_SHAPE = (48, 48, 3)   

model_aug = Sequential() # Establishing networks as sequential
model_aug.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE)) #add convolution layer with 32 filters
model_aug.add(Activation('relu')) 
model_aug.add(Conv2D(32, (3, 3))) #add convolution layer with 32 filters
model_aug.add(Activation('relu')) # add activation function as relu
model_aug.add(MaxPooling2D(pool_size=(2, 2)))
model_aug.add(BatchNormalization())

model_aug.add(Conv2D(64, (3, 3))) #add convolution layer with 64 filters
model_aug.add(Activation('relu')) # add activation function as relu 
model_aug.add(Conv2D(64, (3, 3))) #add convolution layer with 64 filters
model_aug.add(Activation('relu')) # add activation function as relu
model_aug.add(MaxPooling2D(pool_size=(2, 2)))
model_aug.add(BatchNormalization())

model_aug.add(Conv2D(128, (3, 3))) #add convolution layer with 128 filters
model_aug.add(Activation('relu')) # add activation function as relu
model_aug.add(Conv2D(128, (3, 3))) #add convolution layer with 128 filters
model_aug.add(Activation('relu')) # add activation function as relu
model_aug.add(MaxPooling2D(pool_size=(2, 2)))
model_aug.add(BatchNormalization())

model_aug.add(Flatten())
model_aug.add(Dense(128)) #add dense layer with 128 nodes
model_aug.add(Activation('relu')) # add activation function as relu
model_aug.add(Dropout(0.25))
model_aug.add(Dense(7)) #add dense layer with 1 node
model_aug.add(Activation('softmax')) # add activation function as sigmoid as it is better suited for binary outcomes

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model_aug.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
print(model_aug.summary())    

In [ ]:
history_aug=model_aug.fit_generator(      #Fitting the model
        train_data,
        epochs=2,
        validation_data=validation_data,
        )                                        

In [ ]:
evaluation = model_aug.evaluate(test_data) # Creating the evalutaion variable for x_test and y_test_cat
print('Test Accuracy: {}'.format(evaluation[1])) 


In [ ]:
#Plotting loss function with training set and validaton set
plt.subplot(2, 2, 1) # First plot of the two plots
plt.plot(history_aug.history['loss'], label='Loss') # First input variable for line graph(loss)
plt.plot(history_aug.history['val_loss'], label='val_Loss') # second input variable for line graph(val_loss)
plt.legend()
plt.title('Loss evolution')


#Plotting accuracy with training and validation set
plt.subplot(2, 2, 2) 
plt.plot(history_aug.history['accuracy'], label='accuracy') # First input variable for line graph(accuracy)
plt.plot(history_aug.history['val_accuracy'], label='val_accuracy') # second input variable for line graph(val_accuracy)
plt.legend()
plt.title('Accuracy evolution')

In [ ]:
model_aug.save('new_model.h5')  # Saving the model
new_model= tf.keras.models.load_model('new_model.h5')  # Loading the model

### Emotion detection on a single image

In [ ]:
img= cv2.imread('C:/Users/Abhay/images.jpg')  # Taking a smaple image
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Converting it to RGB from BRG

In [ ]:
Prediction = DeepFace.analyze(img)   # Predicting the emotion
Prediction['dominant_emotion']

In [ ]:
faceCascade= cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') # Conveting image into gray scale for HaarCascade
gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces= faceCascade.detectMultiScale(gray,1.1,4)

In [ ]:
for(x,y,w,h) in faces:      # Plotting a rectangle aroung the face
    cv2.rectangle(img, (x,y), (y+w, y+h), (0,225,0), 2)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))    # Ploting the face in RGB with rectangle

### Implimenting it on the live feed

In [ ]:
faceCascade= cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
cap=cv2.VideoCapture(1)       
# Checking if webcam is opened properly
if not cap.isOpened():    
    cap=cv2.VideoCapture(0)

if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
while True:
    # Reading a single image from the video
    ret,frame= cap.read()
    result= DeepFace.analyze(frame,actions=['emotion'])
    # Convert the image into gray scale 
    gray= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    faces= faceCascade.detectMultiScale(gray, 1.1,4)
    
    # Draw rectangle around faces
    for(x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)

    font= cv2.FONT_HERSHEY_SIMPLEX
    #Add the text
    cv2.putText(frame,
               result['dominant_emotion'],
               (50,50),
               font,3,
               (0,0,255),
               2,
               cv2.LINE_4)
    cv2.imshow('Orignal video', frame)
    
    if cv2.waitKey(2) & 0xff == ord ('q'):
            break
        
cap.release()
cv2.destroyAllWindows()
    